In [ ]:
import time
import json
from kafka import KafkaConsumer
from datetime import datetime
import matplotlib.pyplot as plt
import threading

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly




In [ ]:
brokers='localhost:9092'
topic='accident_tmp_input'
sleep_time=300
offset='earliest'

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=brokers, auto_offset_reset=offset,consumer_timeout_ms=1000)
consumer.subscribe([topic])

In [ ]:
msg_array = []
# Create an empty RDD
columns = []
iteration = 0
first_update = True
# Create empty schema



In [ ]:
len(columns)

Initialize a Panda Dataframe batch consumer

In [ ]:
countDF = pd.DataFrame()

Draw plots periodically

In [ ]:


def plot_update(accidents):
    global first_update, fig, iteration
    answerDF = accidents.dropna()

    answerDF["End_Lat"] = pd.to_numeric(answerDF["End_Lat"])
    answerDF["End_Lng"] = pd.to_numeric(answerDF["End_Lng"])
    answerDF['Count']=1
    

    if first_update:
        fig = px.density_mapbox(answerDF, lat='End_Lat', lon='End_Lng', z='Count',mapbox_style="stamen-terrain")
        first_update = False
    else:
        fig.add_densitymapbox(lat=answerDF['End_Lat'], lon=answerDF['End_Lng'], z=answerDF['Count'])

    heatmap_file_name = f'heatmaps.html'
    plotly.offline.plot(fig, filename=heatmap_file_name)
    iteration += 1

Start consuming in a parallel thread

In [ ]:

# Create an empty RDD with empty schema
def parallel_fetch(): 
    while(True):
        for message in consumer:
            global columns, msg_array
            res = message.value.decode('UTF-8').strip('][').split(', ')
            res = [i.replace('"', '') for i in res]
            if len(columns) == 0:
                columns = res
                continue


            msg_array.append(res)
            if(len(msg_array) == 100000):
                main_dataframe = pd.DataFrame(msg_array, columns =columns)
                msg_array = []
                plot_update(main_dataframe)
        
           
                
            
                
                   
            
t1 = threading.Thread(target=parallel_fetch)
t1.start()
    

In [ ]:
len(msg_array)

In [ ]:
msg_array